In [2]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import calendar

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [3]:
from xgboost.sklearn import XGBRegressor
from sklearn.datasets import make_classification
import xgboost as xgb
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
from collections import OrderedDict

/home/alfonso/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/alfonso/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:

# %timeit sirve para evaluar el tiempo de ejecucion
train_x = pd.read_csv('train_paso_3.csv')
test_x = pd.read_csv('test_paso_3.csv')
test = pd.read_csv('trocafone_kaggle_test.csv')

 

In [5]:
train_x = train_x.drop('person', 1)

NameError: name 'train_entrenamiento' is not defined

In [7]:
train_x.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,...,Sony,iPad,iPhone,Direct,Email,Organic,Paid,Referral,Social,desconocido_channel
0,0008ed71,0.00,0.00,3.00,0.00,1.00,0.00,0.00,0.00,0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00
1,000c79fe,1.00,0.00,1.00,0.00,1.00,0.00,1.00,9.00,0.00,...,0.00,0.00,4.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
2,001802e4,5.00,0.00,1.00,0.00,4.00,0.00,0.00,4.00,0.00,...,0.00,0.00,5.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
3,0019e639,29.00,165.00,15.00,2.00,28.00,0.00,13.00,11.00,0.00,...,0.00,0.00,3.00,0.00,0.00,2.00,16.00,1.00,0.00,0.00
4,001b0bf9,0.00,1.00,2.00,1.00,1.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [10]:
test_x.head()

,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,...,Sony,iPad,iPhone,Direct,Email,Organic,Paid,Referral,Social,desconocido_channel
0,0.00,0.00,1.00,0.00,1.00,0.00,1.00,1.00,0.00,4.00,...,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
1,29.00,4.00,7.00,0.00,21.00,1.00,0.00,6.00,0.00,404.00,...,0.00,5.00,405.00,56.00,0.00,1.00,21.00,17.00,0.00,0.00
2,1.00,5.00,1.00,0.00,1.00,0.00,2.00,1.00,0.00,13.00,...,0.00,0.00,6.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00
3,15.00,7.00,1.00,0.00,20.00,0.00,26.00,1.00,5.00,739.00,...,22.00,0.00,8.00,1.00,0.00,14.00,6.00,1.00,0.00,0.00
4,14.00,8.00,2.00,0.00,14.00,0.00,13.00,9.00,0.00,177.00,...,0.00,0.00,176.00,2.00,0.00,3.00,6.00,9.00,0.00,0.00


In [11]:
#feature engieri del train,misma columna train,pero sin label.

In [12]:
#hash
#Transformo texto a numeros
train_x['most_frequent_event'] = train_x.apply(lambda row:  hash(row['most_frequent_event'])\
                        if not(pd.isnull(row['most_frequent_event'])) else np.nan, axis = 1)
train_x['less_frequent_event'] = train_x.apply(lambda row:  hash(row['less_frequent_event'])\
                        if not(pd.isnull(row['less_frequent_event'])) else np.nan, axis = 1)

In [13]:
train_x.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,...,Sony,iPad,iPhone,Direct,Email,Organic,Paid,Referral,Social,desconocido_channel
0,0008ed71,0.00,0.00,3.00,0.00,1.00,0.00,0.00,0.00,0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00
1,000c79fe,1.00,0.00,1.00,0.00,1.00,0.00,1.00,9.00,0.00,...,0.00,0.00,4.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
2,001802e4,5.00,0.00,1.00,0.00,4.00,0.00,0.00,4.00,0.00,...,0.00,0.00,5.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
3,0019e639,29.00,165.00,15.00,2.00,28.00,0.00,13.00,11.00,0.00,...,0.00,0.00,3.00,0.00,0.00,2.00,16.00,1.00,0.00,0.00
4,001b0bf9,0.00,1.00,2.00,1.00,1.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [14]:
train_y = train_x['label']
train_y.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [45]:
train_x = train_x.drop(['label','person'], 1)
train_x.head()

,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,...,Sony,iPad,iPhone,Direct,Email,Organic,Paid,Referral,Social,desconocido_channel
0,0.00,0.00,3.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00
1,1.00,0.00,1.00,0.00,1.00,0.00,1.00,9.00,0.00,3.00,...,0.00,0.00,4.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
2,5.00,0.00,1.00,0.00,4.00,0.00,0.00,4.00,0.00,4.00,...,0.00,0.00,5.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
3,29.00,165.00,15.00,2.00,28.00,0.00,13.00,11.00,0.00,189.00,...,0.00,0.00,3.00,0.00,0.00,2.00,16.00,1.00,0.00,0.00
4,0.00,1.00,2.00,1.00,1.00,0.00,0.00,0.00,0.00,2.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [46]:
modelo_xgb = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, gamma = 0 , n_estimators = 1000)

In [50]:
modelo_xgb.fit(train_x,train_y)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [51]:
OrderedDict(sorted(modelo_xgb.booster().get_fscore().items(), key=lambda t: t[1], reverse=True))

OrderedDict([('mean_per_event', 1666),
             ('desconocido_modelo', 1185),
             ('viewed product', 1167),
             ('brand listing', 1027),
             ('ad campaign hit', 1004),
             ('generic listing', 985),
             ('checkout', 923),
             ('16GB', 919),
             ('Bom', 915),
             ('Samsung', 907),
             ('Muito Bom', 891),
             ('count_most_frequent_event', 885),
             ('32GB', 860),
             ('searched products', 836),
             ('iPhone', 835),
             ('Excelente', 796),
             ('visited site', 756),
             ('search engine hit', 726),
             ('64GB', 663),
             ('8GB', 645),
             ('Motorola', 632),
             ('less_frequent_event', 630),
             ('128GB', 577),
             ('Paid', 536),
             ('Direct', 509),
             ('count_less_frequent_event', 434),
             ('conversion', 428),
             ('Bom - Sem Touch ID', 426),
           

In [93]:
train_x = train_x.drop(['lead','Quantum','Email','desconocido_channel'],axis=1)

In [94]:
#parameters_for_testing = {
#    'colsample_bytree':[0.4,0.6,0.8],
#    'gamma':[0,0.03,0.1],
#    'min_child_weight':[1.5,6,10],
#    'learning_rate':[0.1,0.07,0.01],
#    'max_depth':[3,5,7],
#    'n_estimators':[10,100,1000],
#    'reg_alpha':[1e-5, 1e-2,  0.75],
#    'reg_lambda':[1e-5, 1e-2, 0.45],
#    'subsample':[0.6,0.95]
#}

In [95]:
#gsearch1 = GridSearchCV(estimator = modelo_xgb, param_grid = parameters_for_testing,\
#                        n_jobs=6,iid=False, verbose=10,scoring='roc_auc')
#gsearch1.fit(train_x,train_y)
#print (gsearch1.grid_scores_)
#print('best params')
#print (gsearch1.best_params_)
#print('best score')
#print (gsearch1.best_score_)

In [96]:
# respuesta del gridsearch:
#colsample_bytree=0.4, gamma=0, learning_rate=0.07, max_depth=3, min_child_weight=6,
#n_estimators=100, reg_alpha=0.01, reg_lambda=1e-05, subsample=0.95, score=0.840199

In [97]:
#creo el mejor modelo:
mejor_modelo_xgb = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.4, learning_rate = 0.07,
                max_depth = 3, gamma = 0 , n_estimators = 100, min_child_weight=6,
                reg_alpha=0.01, reg_lambda=1e-05, subsample=0.95)

In [98]:
#entreno el mejor modelo
mejor_modelo_xgb.fit(train_x,train_y)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.4,
       gamma=0, learning_rate=0.07, max_delta_step=0, max_depth=3,
       min_child_weight=6, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0.01, reg_lambda=1e-05,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.95)

In [99]:
#hash del test
#hash
#Transformo texto a numeros
test_x['most_frequent_event'] = test_x.apply(lambda row:  hash(row['most_frequent_event'])\
                        if not(pd.isnull(row['most_frequent_event'])) else np.nan, axis = 1)
test_x['less_frequent_event'] = test_x.apply(lambda row:  hash(row['less_frequent_event'])\
                        if not(pd.isnull(row['less_frequent_event'])) else np.nan, axis = 1)

In [100]:
test_x = test_x.drop(['lead','Quantum','Email','desconocido_channel'],axis=1)

In [101]:
test_x.head()

,ad campaign hit,brand listing,checkout,conversion,generic listing,search engine hit,searched products,staticpage,viewed product,visited site,...,Motorola,Samsung,Sony,iPad,iPhone,Direct,Organic,Paid,Referral,Social
0,0.00,0.00,1.00,0.00,1.00,1.00,1.00,0.00,4.00,1.00,...,0.00,5.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
1,29.00,4.00,7.00,0.00,21.00,0.00,6.00,0.00,404.00,95.00,...,0.00,2.00,0.00,5.00,405.00,56.00,1.00,21.00,17.00,0.00
2,1.00,5.00,1.00,0.00,1.00,2.00,1.00,0.00,13.00,2.00,...,8.00,0.00,0.00,0.00,6.00,0.00,1.00,1.00,0.00,0.00
3,15.00,7.00,1.00,0.00,20.00,26.00,1.00,5.00,739.00,22.00,...,517.00,34.00,22.00,0.00,8.00,1.00,14.00,6.00,1.00,0.00
4,14.00,8.00,2.00,0.00,14.00,13.00,9.00,0.00,177.00,20.00,...,0.00,3.00,0.00,0.00,176.00,2.00,3.00,6.00,9.00,0.00


In [102]:
#predecimos
prediction = mejor_modelo_xgb.predict(test_x)

In [103]:
ids = test['person'].values
dataKaggle = pd.DataFrame(ids ,columns={'person'})
dataKaggle.head(5)

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [104]:
#Aca agregamos una columna que seria se postulop
dataKaggle['label'] = prediction

In [105]:
dataKaggle.nlargest(5,'label')

,person,label
7794,62147633,0.66
262,2b213b2c,0.65
14034,f6c7886d,0.65
18722,3ae22242,0.52
4309,5059f7fd,0.49


In [106]:
dataKaggle['label'] = dataKaggle['label'].transform(lambda x: 0.0 if x < 0.0 else x)

In [107]:
dataKaggle.to_csv("predictionXGBoost.csv", index = False)